In [ ]:
import os 
import datetime
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as trv
from PIL import Image
import random
import utils
import logging
import utils

In [ ]:
VOC_DIR = "dataset/VOCdevkit/VOC2012/"
NUM_CLASSES = 21
WEIGHT_PATH = "weights/08062258.pth"

In [ ]:
Valid_dataset = utils.VOC_dataset(VOC_DIR, train=False)
model = utils.ResNet18_FCN(NUM_CLASSES)
model.load_state_dict(torch.load(WEIGHT_PATH))
model.eval();

In [ ]:
image, label = next(iter(Valid_dataset))

inputs = torch.unsqueeze(image, 0)
output = model(inputs).argmax(dim=1)[0]
mask = utils.label2image(output).astype(np.uint8)

std  = torch.Tensor([0.229, 0.224, 0.225]).unsqueeze(1).unsqueeze(2)
mean = torch.Tensor([0.485, 0.456, 0.406]).unsqueeze(1).unsqueeze(2)
image = (image * std + mean) * 255.
image_display = torch.clip(image.permute(1, 2, 0), min=0.0, max=255.).numpy().astype(np.uint8)

superimposed_img = mask.astype(np.float32) * 0.2 + image_display.astype(np.float32) * 0.8
superimposed_img = np.uint8(np.clip(superimposed_img, 0, 255))

In [ ]:
fig, ax = plt.subplots(1, 3, dpi=200, facecolor=(1, 1, 1))

axx = plt.subplot(1, 3, 1)
plt.imshow(image_display)
plt.axis("off")

axx = plt.subplot(1, 3, 2)
plt.imshow(mask)
plt.axis("off")

axx = plt.subplot(1, 3, 3)
plt.imshow(superimposed_img)
plt.axis("off")

plt.tight_layout()
fig.subplots_adjust(top=1.5)
fig.suptitle("Sample", fontsize=15)

plt.show()